In [31]:
# -*- coding: utf-8 -*-
import sqlite3
import requests
import json
import time
import datetime
from selenium import webdriver
import re

class FacebookProcess(object):
    
    
    def __init__(self, email, password):
        self.login(email, password)
        self.fb_version = 'v2.9'
        self.fb_dtsg = self.driver.find_element_by_name("fb_dtsg").get_property("value")
        self.cookie = '; '.join(['{}={}'.format(item.get('name'), item.get('value'))for item in self.driver.get_cookies()])
        self.token = self.get_access_token()
        
    def login(self, email, password):
        self.driver = webdriver.Firefox()
        self.driver.get('https://zh-tw.facebook.com/login')
        time.sleep(3)
        self.driver.find_element_by_id("pass").clear()
        self.driver.find_element_by_id("pass").send_keys(password)
        self.driver.find_element_by_id("email").clear()
        self.driver.find_element_by_id("email").send_keys(email)
        self.driver.find_element_by_id("loginbutton").click()
        time.sleep(3)
    
    def get_access_token(self):
        headers = {
        'cookie':self.cookie
        }
        res = requests.get('https://developers.facebook.com/tools/explorer/', headers=headers)
        token = re.findall(r'\"accessToken\":\"([A-Za-z0-9_]*)\"', res.text)[0]
        return token
        
    def like_post(self, post_id):
        data2 = 'client_id=1494422643451:996525474&ft_ent_identifier={}&reaction_type=1&fb_dtsg={}'.format(post_id.split('_')[1], self.fb_dtsg)
        headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'referer':'https://www.facebook.com/',
        'cookie':self.cookie
        }

        requests.post('https://www.facebook.com/ufi/reaction/?dpr=1', data=data2, headers=headers)
        
    def like_post2(self, post_id):
        self.driver.get('https://www.facebook.com/{}'.format(post_id))
        time.sleep(5)
        try: self.driver.find_element_by_link_text(u"讚").click()
        except: self.driver.find_element_by_xpath(u"(//a[contains(text(),'讚')])[12]").click()
            
    def like_page(self, page_id):
        self.driver.get('https://www.facebook.com/{}'.format(page_id))
        time.sleep(5)
        self.driver.find_element_by_xpath("(//button[@value='1'])[2]").click()

    def share_post(self, link):
        attachment = {
        "link": link,
        "privacy": "{'value':'EVERYONE'}",
        }
        requests.post('https://graph.facebook.com/{}/me/feed?access_token={}'.format(self.fb_version, self.token), data=attachment)

    def fblink_to_post_id(self, fb_link):
        post_id = fb_link.split('/')[-1]
        page_name = fb_link.split('/')[3]
        res = requests.get('https://graph.facebook.com/{}/{}?access_token={}'.format(self.fb_version, page_name, self.token))
        page_id = json.loads(res.text)['id']
        
        return page_id + '_' + post_id
    
    def post_comment(self, post_id, message, tags):
        tags = ''.join(["@[{}:{}]".format(key, value) for key, value in tags.items()])
        data2 = """ft_ent_identifier={}&comment_text={}{}
        &source=21&client_id=1493913966456:1963482973&fb_dtsg={}""".format(post_id.split('_')[1], tags, message.encode('utf8'), self.fb_dtsg)
        headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36', 
        'referer':'https://www.facebook.com/',
        'cookie':self.cookie
        }
        
        requests.post("https://www.facebook.com/ufi/add/comment/?dpr=1", data=data2, headers=headers)


if __name__ == "__main__":
    
    with open(r'C:\Users\Lenovo\Desktop\fb.txt', 'r') as f:
        f = list(f)
        email = f[0]
        password = f[1]
    fb2 = FacebookProcess(email, password)
    today = datetime.datetime.now().strftime("%Y/%m/%d")
    conn = sqlite3.connect('..\luckydog.sqlite')
    cur = conn.cursor()
    cur.execute('select * from luckydog where is_solved=0 and deadline="{}";'.format(today))
    for row in cur.fetchall():
        fb_link = row[0]
        if ('https://www.facebook.com' in fb_link) & ('posts' in fb_link) & (row[4]==0):
            '''
            #抓取貼文第 n 則留言並替換Tag的好友後留言
            post_id = fb2.fblink_to_post_id(fb_link)
            try:
                res = requests.get('https://graph.facebook.com/{}/{}/comments?limit=20&access_token={}'
                                   .format(fb2.fb_version, post_id, fb2.token))
                text = json.loads(res.text)['data'][-1]['message']
            except Exception as e:
                print 'post_comment error', fb_link
                continue
            tags = {'100000256118186':'高健盛','100005984324342':'劉金花','100009516664068':'廖寶珠'}
            fb2.post_comment(post_id, text, tags=tags)
            try:
                fb2.like_post(post_id)
            except:
                print 'like_post error', fb_link
            try:
                fb2.like_page(post_id.split('_')[0])
            except:
                print 'like_page error', fb_link
            fb2.share_post(fb_link)
            '''
            cur.execute('update luckydog set is_solved=1 where fb_link="{}"'.format(fb_link))
    
    #在luckydog追蹤貼文
    fb2.driver.get('https://zh-tw.facebook.com/login?skip_api_login=1&api_key=100361976797783&signed_next=1&next=https%3A%2F%2Fwww.facebook.com%2Fv2.3%2Fdialog%2Foauth%3Fredirect_uri%3Dhttp%253A%252F%252Fwww.luckydog.tw%252Foauth_callback_fb%26scope%3Demail%26client_id%3D100361976797783%26ret%3Dlogin%26logger_id%3D894c2901-e7c1-663b-68be-ded486fbc488&cancel_url=http%3A%2F%2Fwww.luckydog.tw%2Foauth_callback_fb%3Ferror%3Daccess_denied%26error_code%3D200%26error_description%3DPermissions%2Berror%26error_reason%3Duser_denied%23_%3D_&display=page&locale=zh_TW&logger_id=894c2901-e7c1-663b-68be-ded486fbc488')
    cur.execute('select * from luckydog where deadline="{}" and is_solved=1 and is_followed=0'.format(today))
    for row in cur.fetchall():
        fb2.driver.get(row[2])
        time.sleep(3)
        fb2.driver.find_element_by_css_selector("#diglink1 > div > div").click()
        cur.execute('update luckydog set is_followed=1 where fb_link="{}"'.format(row[0]))
    fb2.driver.quit()
    conn.commit()
    conn.close()
    